In [ ]:
import os
import google.generativeai as genai
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

generation_config = { # config :  구성 및 설정, 생성 AI 모델이 텍스트 생성 시, 사용할 설정 값을 정의한 딕셔너리 
  "temperature": 0,
  "top_k": 1,
  "max_output_tokens": 4000,
}


class LLM:
    def __init__(self, model_name) -> None: # __init__ 객체 생성시 자동으로 호출되는 생성자. # None 은 반환값이 없음을 나타내는 타입 힌트. 
        self.model_name = model_name
        self.model = self.create_model(model_name) # 여기서 self.model = model 이 왜 아니야? 

    def create_model(self, model_name):
        match model_name: # match는 model name 값에 따라 다른 동작을 수행함. 
            case "gemini-pro-vision":
                genai.configure(api_key=os.environ.get("GOOGLE_API_KEY")) 
                return genai.GenerativeModel(model_name)
            case "gemini-pro":
                genai.configure(api_key=os.environ.get("GOOGLE_API_KEY"))
                return genai.GenerativeModel(
                    model_name,generation_config=generation_config)
            case "OpenAI":
                return OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))
            case _: # 기본값으로 작동. 어떤 모델도 매치 하지 않으면 실행 됨. 
                print("Not Implemented")
          
    def __call__(self, prompt, image=None): # __call__은 객체를 함수 처럼 호출하게 하는 특수 매서드임. 이미지 전달 안됐을 경우 None으로 처리리
        if self.model_name == 'gemini-pro-vision':
            response = self.model.generate_content(
                [image, prompt]
            )
        elif self.model_name == "gemini-pro":
            response = self.model.generate_content(
                prompt)
            return response.text
        
        elif self.model_name == 'OpenAI':
            res = self.model.chat.completions.create(
                model="gpt-3.5-turbo-1106",
                # response_format={"type": "json_object"},
                messages=[ 
                    # {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": f"{prompt}"},
                ],
                # seed=10,
                temperature=0
            )
            return res.choices[0].message.content 